<a href="https://colab.research.google.com/github/TaranehPaula/ETL/blob/main/ETL_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas Instaladas

In [ ]:
!pip install pandas

In [ ]:
!pip install pandera

# Bibliotecas Importadas

In [3]:
import pandas as pd

In [4]:
import pandera as pa

# EXTRAÇÃO do DataSet

In [8]:
df = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/ocorrencia.csv',sep=';',parse_dates=['ocorrencia_dia'], dayfirst = True)

In [9]:
df.head(2)

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,52242,52242,52242,52242,52242,INCIDENTE,NaN,NaN,PORTO ALEGRE,RS,...,2012-01-05,20:27:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,45331,45331,45331,45331,45331,ACIDENTE,-23.4355555556,-46.4730555556,GUARULHOS,SP,...,2012-01-06,13:44:00,SIM,FINALIZADA,A-582/CENIPA/2014,SIM,2016-09-01,3,1,NÃO


## VALIDAÇÃO (Data Validation)

# Montagem do Schema do DataFrame com a Pandera

In [10]:
schema = pa.DataFrameSchema(
    columns= {
        'codigo_ocorrencia':pa.Column(pa.Int),
        'codigo_ocorrencia2':pa.Column(pa.Int),
        'ocorrencia_classificacao':pa.Column(pa.String),
        'ocorrencia_cidade':pa.Column(pa.String),
        'ocorrencia_uf':pa.Column(pa.String, pa.Check.str_length(2,2),nullable=True),
        'ocorrencia_aerodromo':pa.Column(pa.String, nullable=True),
        'ocorrencia_dia':pa.Column(pa.DateTime),
        'ocorrencia_hora':pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        'total_recomendacoes':pa.Column(pa.Int)
    }
)

In [17]:
schema.validate(df)

SchemaError: ignored

In [ ]:
df.dtypes # para visualizar os tipos de dados que foram carregados

## LIMPEZA (Data Cleaning)

In [ ]:
schema.validate(df) #Fazer a primeira conferência da validação do Schema

### Backup

In [19]:
df2 = df.copy

In [20]:
# Limpar todas as linhas que contenham '**'
#df.loc[df.ocorrencia_aerodromo == '**',['ocorrencia_aerodromo']] = pd.NA

In [21]:
# limpar todas as inconsistências listadas acima com a função 'replace'

df.replace(['**','###!','####','****','*****','NULL'], pd.NA, inplace=True)

In [22]:
df   

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,52242,52242,52242,52242,52242,INCIDENTE,NaN,NaN,PORTO ALEGRE,RS,...,2012-01-05,20:27:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,45331,45331,45331,45331,45331,ACIDENTE,-23.4355555556,-46.4730555556,GUARULHOS,SP,...,2012-01-06,13:44:00,SIM,FINALIZADA,A-582/CENIPA/2014,SIM,2016-09-01,3,1,NÃO
2,45333,45333,45333,45333,45333,ACIDENTE,***,***,VIAMÃO,RS,...,2012-01-06,13:00:00,NaN,FINALIZADA,A-070/CENIPA/2013,SIM,2013-11-27,0,1,NÃO
3,45401,45401,45401,45401,45401,ACIDENTE,***,***,SÃO SEBASTIÃO,SP,...,2012-01-06,17:00:00,***,NaN,NaN,NÃO,NaN,0,1,NÃO
4,45407,45407,45407,45407,45407,ACIDENTE,***,***,SÃO SEPÉ,RS,...,2012-01-06,16:30:00,SIM,FINALIZADA,A-071/CENIPA/2013,SIM,2013-11-27,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5162,80458,80458,80458,80458,80458,ACIDENTE,-17.999194,-51.642861,JATAÍ,GO,...,2021-12-30,20:30:00,SIM,ATIVA,A DEFINIR,NÃO,NaN,0,1,NÃO
5163,80452,80452,80452,80452,80452,ACIDENTE,-22.585556,-50.753889,MARACAÍ,SP,...,2021-12-31,09:30:00,SIM,ATIVA,A DEFINIR,NÃO,NaN,0,1,NÃO
5164,80457,80457,80457,80457,80457,INCIDENTE GRAVE,-29.695833,-51.081667,NOVO HAMBURGO,RS,...,2021-12-31,11:59:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
5165,80460,80460,80460,80460,80460,INCIDENTE,-25.403333,-49.233611,CURITIBA,PR,...,2021-12-31,15:12:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO


In [23]:
# Saber quantos <NA> eu tenho? Quantos dados não informados eu tenho?

df.isna().sum()

codigo_ocorrencia                    0
codigo_ocorrencia1                   0
codigo_ocorrencia2                   0
codigo_ocorrencia3                   0
codigo_ocorrencia4                   0
ocorrencia_classificacao             0
ocorrencia_latitude                977
ocorrencia_longitude               978
ocorrencia_cidade                    0
ocorrencia_uf                        0
ocorrencia_pais                      0
ocorrencia_aerodromo              1905
ocorrencia_dia                       0
ocorrencia_hora                      1
investigacao_aeronave_liberada     166
investigacao_status                257
divulgacao_relatorio_numero        391
divulgacao_relatorio_publicado       0
divulgacao_dia_publicacao         3816
total_recomendacoes                  0
total_aeronaves_envolvidas           0
ocorrencia_saida_pista               0
dtype: int64

In [24]:
df.isnull().sum()

codigo_ocorrencia                    0
codigo_ocorrencia1                   0
codigo_ocorrencia2                   0
codigo_ocorrencia3                   0
codigo_ocorrencia4                   0
ocorrencia_classificacao             0
ocorrencia_latitude                977
ocorrencia_longitude               978
ocorrencia_cidade                    0
ocorrencia_uf                        0
ocorrencia_pais                      0
ocorrencia_aerodromo              1905
ocorrencia_dia                       0
ocorrencia_hora                      1
investigacao_aeronave_liberada     166
investigacao_status                257
divulgacao_relatorio_numero        391
divulgacao_relatorio_publicado       0
divulgacao_dia_publicacao         3816
total_recomendacoes                  0
total_aeronaves_envolvidas           0
ocorrencia_saida_pista               0
dtype: int64

In [25]:
# Colocar um 'valor' no meu dado <NA> com a função fillna(valor, inplace=True)
df.fillna(0)

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,52242,52242,52242,52242,52242,INCIDENTE,0,0,PORTO ALEGRE,RS,...,2012-01-05,20:27:00,***,FINALIZADA,***,NÃO,0,0,1,NÃO
1,45331,45331,45331,45331,45331,ACIDENTE,-23.4355555556,-46.4730555556,GUARULHOS,SP,...,2012-01-06,13:44:00,SIM,FINALIZADA,A-582/CENIPA/2014,SIM,2016-09-01,3,1,NÃO
2,45333,45333,45333,45333,45333,ACIDENTE,***,***,VIAMÃO,RS,...,2012-01-06,13:00:00,0,FINALIZADA,A-070/CENIPA/2013,SIM,2013-11-27,0,1,NÃO
3,45401,45401,45401,45401,45401,ACIDENTE,***,***,SÃO SEBASTIÃO,SP,...,2012-01-06,17:00:00,***,0,0,NÃO,0,0,1,NÃO
4,45407,45407,45407,45407,45407,ACIDENTE,***,***,SÃO SEPÉ,RS,...,2012-01-06,16:30:00,SIM,FINALIZADA,A-071/CENIPA/2013,SIM,2013-11-27,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5162,80458,80458,80458,80458,80458,ACIDENTE,-17.999194,-51.642861,JATAÍ,GO,...,2021-12-30,20:30:00,SIM,ATIVA,A DEFINIR,NÃO,0,0,1,NÃO
5163,80452,80452,80452,80452,80452,ACIDENTE,-22.585556,-50.753889,MARACAÍ,SP,...,2021-12-31,09:30:00,SIM,ATIVA,A DEFINIR,NÃO,0,0,1,NÃO
5164,80457,80457,80457,80457,80457,INCIDENTE GRAVE,-29.695833,-51.081667,NOVO HAMBURGO,RS,...,2021-12-31,11:59:00,SIM,FINALIZADA,***,NÃO,0,0,1,NÃO
5165,80460,80460,80460,80460,80460,INCIDENTE,-25.403333,-49.233611,CURITIBA,PR,...,2021-12-31,15:12:00,SIM,FINALIZADA,***,NÃO,0,0,1,NÃO


In [26]:
# Colocar um valor em uma coluna específica que tenha um <NA> com a função fillna
# df.fillna(value={'total_recomendacoes':10}, inplace=True)

In [ ]:
# Excluir uma coluna / DROPAR que não faz sentido para a Análise
# importante: axis = 1 é coluna e axis = 0 é a linha

df.drop(['coluna1'], axis=1, inplace= True)


In [28]:
# DROPAR /EXCLUIR TODOS OS VALORES <NA>
#df.dropna(inplace=True)

In [29]:
# DROPAR /EXCLUIR O <NA> POR COLUNA
df.dropna(subset=['ocorrencia_uf'], inplace=True)

### Tratar valores duplicados

In [30]:
# DROPAR VALORES DUPLICADOS
df.drop_duplicates(inplace=True)

# TRANSFORMAÇÃO (Data Transforming)

In [31]:
df.codigo_ocorrencia.is_unique

True

### informando ao python que o código da ocorrência vai passar a ser o indice do DataFrame

In [32]:
#df.set_index('codigo_ocorrencia', inplace=True)  

### Resetando um índice alterado

In [33]:
#df.reset_index(drop=True,inplace=True)

### Filtros

In [34]:
df.loc[df.ocorrencia_uf.isnull()]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista


In [35]:
df.loc[df.ocorrencia_aerodromo.isnull()]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
2,45333,45333,45333,45333,45333,ACIDENTE,***,***,VIAMÃO,RS,...,2012-01-06,13:00:00,NaN,FINALIZADA,A-070/CENIPA/2013,SIM,2013-11-27,0,1,NÃO
3,45401,45401,45401,45401,45401,ACIDENTE,***,***,SÃO SEBASTIÃO,SP,...,2012-01-06,17:00:00,***,NaN,NaN,NÃO,NaN,0,1,NÃO
4,45407,45407,45407,45407,45407,ACIDENTE,***,***,SÃO SEPÉ,RS,...,2012-01-06,16:30:00,SIM,FINALIZADA,A-071/CENIPA/2013,SIM,2013-11-27,0,1,NÃO
5,52243,52243,52243,52243,52243,INCIDENTE,NaN,NaN,UBATUBA,SP,...,2012-01-06,14:30:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
8,45391,45391,45391,45391,45391,ACIDENTE,-19.9133333333,-48.2930555556,CONCEIÇÃO DAS ALAGOAS,MG,...,2012-01-08,16:00:00,***,FINALIZADA,A DEFINIR,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5151,80446,80446,80446,80446,80446,ACIDENTE,-9.13611,-57.05444,JACAREACANGA,PA,...,2021-12-20,17:10:00,SIM,ATIVA,A DEFINIR,NÃO,NaN,0,1,NÃO
5155,80450,80450,80450,80450,80450,ACIDENTE,-22.09388889,-55.37583333,DOURADOS,MS,...,2021-12-28,10:30:00,SIM,ATIVA,A DEFINIR,NÃO,NaN,0,1,SIM
5159,80455,80455,80455,80455,80455,INCIDENTE GRAVE,-7.04638,-34.91277,SANTA RITA,PB,...,2021-12-29,18:50:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,SIM
5162,80458,80458,80458,80458,80458,ACIDENTE,-17.999194,-51.642861,JATAÍ,GO,...,2021-12-30,20:30:00,SIM,ATIVA,A DEFINIR,NÃO,NaN,0,1,NÃO


In [36]:
df.loc[df.ocorrencia_hora.isnull()]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
2974,78823,78823,78823,78823,78823,INCIDENTE,***,***,CORUMBÁ,MS,...,2017-07-09,NaN,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO


In [37]:
df.count()

codigo_ocorrencia                 5167
codigo_ocorrencia1                5167
codigo_ocorrencia2                5167
codigo_ocorrencia3                5167
codigo_ocorrencia4                5167
ocorrencia_classificacao          5167
ocorrencia_latitude               4190
ocorrencia_longitude              4189
ocorrencia_cidade                 5167
ocorrencia_uf                     5167
ocorrencia_pais                   5167
ocorrencia_aerodromo              3262
ocorrencia_dia                    5167
ocorrencia_hora                   5166
investigacao_aeronave_liberada    5001
investigacao_status               4910
divulgacao_relatorio_numero       4776
divulgacao_relatorio_publicado    5167
divulgacao_dia_publicacao         1351
total_recomendacoes               5167
total_aeronaves_envolvidas        5167
ocorrencia_saida_pista            5167
dtype: int64

In [38]:
# Quero saber as ocorrencias com mais de 10 recomendações

filtro_recomendacoes = df.total_recomendacoes >10
df.loc[filtro_recomendacoes]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
844,47938,47938,47938,47938,47938,INCIDENTE,-15.8691666667,-47.9208333333,BRASÍLIA,DF,...,2013-04-13,18:00:00,***,FINALIZADA,I-067/CENIPA/2013,SIM,2013-07-10,11,1,NÃO
1669,52265,52265,52265,52265,52265,ACIDENTE,-23.9597222222,-46.3269444444,SANTOS,SP,...,2014-08-13,13:03:00,NaN,FINALIZADA,A-134/CENIPA/2014,SIM,2016-01-11,13,1,NÃO
2804,66432,66432,66432,66432,66432,INCIDENTE GRAVE,-20.3861111111,-40.4888888889,VITÓRIA,ES,...,2017-02-21,11:47:00,NaN,FINALIZADA,IG-031/CENIPA/2017,SIM,2020-10-02,12,1,NÃO


In [39]:
# Quero saber as cidades com as ocorrencias com mais de 10 recomendações 
filtro_recomendacoes = df.total_recomendacoes >10
df.loc[filtro_recomendacoes,'ocorrencia_cidade']

844     BRASÍLIA
1669      SANTOS
2804     VITÓRIA
Name: ocorrencia_cidade, dtype: object

In [40]:
# Quero saber as cidades com as ocorrencias com mais de 10 recomendações e o total de recomendações em cada cidade
filtro_recomendacoes = df.total_recomendacoes >10
df.loc[filtro_recomendacoes,['ocorrencia_cidade','total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
844,BRASÍLIA,11
1669,SANTOS,13
2804,VITÓRIA,12


In [42]:
# Filtrar as ocorrências do tipo Grave
filtro_graves = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro_graves]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
6,50713,50713,50713,50713,50713,INCIDENTE GRAVE,***,***,CAMPINAS,SP,...,2012-01-07,18:15:00,SIM,FINALIZADA,IG-184/CENIPA/2013,SIM,2014-01-08,0,1,NÃO
12,45396,45396,45396,45396,45396,INCIDENTE GRAVE,-5.3677777778,-49.1383333333,MARABÁ,PA,...,2012-01-11,11:21:00,SIM,FINALIZADA,IG-508/CENIPA/2017,SIM,2017-05-02,0,1,NÃO
18,50714,50714,50714,50714,50714,INCIDENTE GRAVE,-2.8966666667,-40.8569444444,CAMOCIM,CE,...,2012-01-17,18:25:00,SIM,FINALIZADA,IG-073/CENIPA/2013,SIM,2014-10-01,0,1,NÃO
19,45399,45399,45399,45399,45399,INCIDENTE GRAVE,-1.1966666667,-49.2675,MUANÁ,PA,...,2012-01-18,17:35:00,SIM,FINALIZADA,IG-074/CENIPA/2013,SIM,2014-08-15,0,1,NÃO
24,45582,45582,45582,45582,45582,INCIDENTE GRAVE,-16.5852777778,-54.7241666667,RONDONÓPOLIS,MT,...,2012-01-20,17:00:00,SIM,FINALIZADA,IG-535/CENIPA/2015,SIM,2015-07-21,0,1,SIM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5139,80427,80427,80427,80427,80427,INCIDENTE GRAVE,-10.022222,-58.237222,NOVA BANDEIRANTES,MT,...,2021-12-12,17:20:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,SIM
5146,80448,80448,80448,80448,80448,INCIDENTE GRAVE,-11.895278,-49.692222,FORMOSO DO ARAGUAIA,TO,...,2021-12-18,19:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,SIM
5158,80454,80454,80454,80454,80454,INCIDENTE GRAVE,-21.144167,-50.426389,ARAÇATUBA,SP,...,2021-12-29,21:35:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,SIM
5159,80455,80455,80455,80455,80455,INCIDENTE GRAVE,-7.04638,-34.91277,SANTA RITA,PB,...,2021-12-29,18:50:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,SIM


In [41]:
# Filtrar as ocorrências do tipo Grave no Estado de São Paulo
filtro_graves = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro_SP = df.ocorrencia_uf == 'SP'
df.loc[filtro_graves & filtro_SP]   # Usamos o operador & (e) para juntar os dois

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
6,50713,50713,50713,50713,50713,INCIDENTE GRAVE,***,***,CAMPINAS,SP,...,2012-01-07,18:15:00,SIM,FINALIZADA,IG-184/CENIPA/2013,SIM,2014-01-08,0,1,NÃO
78,45598,45598,45598,45598,45598,INCIDENTE GRAVE,-22.979167,-46.537500,BRAGANÇA PAULISTA,SP,...,2012-02-20,14:10:00,***,FINALIZADA,A DEFINIR,NÃO,NaN,0,1,SIM
86,45653,45653,45653,45653,45653,INCIDENTE GRAVE,-23.435556,-46.473056,GUARULHOS,SP,...,2012-02-24,20:04:00,***,FINALIZADA,A DEFINIR,NÃO,NaN,0,1,NÃO
91,45599,45599,45599,45599,45599,INCIDENTE GRAVE,-22.9791666667,-46.5375,BRAGANÇA PAULISTA,SP,...,2012-02-27,19:15:00,SIM,FINALIZADA,IG-087/CENIPA/2013,SIM,2014-10-14,0,1,NÃO
140,45651,45651,45651,45651,45651,INCIDENTE GRAVE,-23.379722,-47.384444,ITU,SP,...,2012-03-24,20:45:00,***,FINALIZADA,IG-593/CENIPA/2018,SIM,2021-08-19,1,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4954,80200,80200,80200,80200,80200,INCIDENTE GRAVE,-23.005278,-46.636944,BRAGANÇA PAULISTA,SP,...,2021-07-25,12:25:00,SIM,FINALIZADA,A DEFINIR,NÃO,NaN,0,1,NÃO
4980,80238,80238,80238,80238,80238,INCIDENTE GRAVE,-20.457222,-50.002500,VOTUPORANGA,SP,...,2021-08-11,19:09:00,SIM,ATIVA,IG-098/CENIPA/2021,NÃO,NaN,0,1,NÃO
5003,80265,80265,80265,80265,80265,INCIDENTE GRAVE,-23.51000,-46.63000,SÃO PAULO,SP,...,2021-08-31,16:50:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
5098,80382,80382,80382,80382,80382,INCIDENTE GRAVE,-22.71000,-47.62000,PIRACICABA,SP,...,2021-11-16,20:50:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,SIM


In [43]:
# Filtrar as ocorrências do tipo Incidente Grave ou Incidente no Estado de São Paulo
filtro_graves = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE') # usamos o opoerador | 'ou'
filtro_SP = df.ocorrencia_uf == 'SP'
df.loc[filtro_graves & filtro_SP]   # Usamos o operador & para juntar os dois

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
5,52243,52243,52243,52243,52243,INCIDENTE,NaN,NaN,UBATUBA,SP,...,2012-01-06,14:30:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
6,50713,50713,50713,50713,50713,INCIDENTE GRAVE,***,***,CAMPINAS,SP,...,2012-01-07,18:15:00,SIM,FINALIZADA,IG-184/CENIPA/2013,SIM,2014-01-08,0,1,NÃO
28,52248,52248,52248,52248,52248,INCIDENTE,NaN,NaN,SÃO PAULO,SP,...,2012-01-21,11:15:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
35,52054,52054,52054,52054,52054,INCIDENTE,NaN,NaN,SÃO PAULO,SP,...,2012-01-25,23:00:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
36,52055,52055,52055,52055,52055,INCIDENTE,NaN,NaN,RIBEIRÃO PRETO,SP,...,2012-01-25,09:30:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5134,80425,80425,80425,80425,80425,INCIDENTE,-23.626111,-46.656389,SÃO PAULO,SP,...,2021-12-09,20:30:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
5143,80431,80431,80431,80431,80431,INCIDENTE,-23.9280,-46.2997,SANTOS,SP,...,2021-12-15,12:30:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
5157,80453,80453,80453,80453,80453,INCIDENTE,-23.00694444444,-47.13444444444,CAMPINAS,SP,...,2021-12-29,09:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
5158,80454,80454,80454,80454,80454,INCIDENTE GRAVE,-21.144167,-50.426389,ARAÇATUBA,SP,...,2021-12-29,21:35:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,SIM


In [44]:
# 2ª forma de Filtrar as ocorrências do tipo Incidente Grave ou Incidente no Estado de São Paulo, usando a função ''.isin'
filtro_graves = df.ocorrencia_classificacao.isin == (['INCIDENTE GRAVE', 'INCIDENTE']) 
filtro_SP = df.ocorrencia_uf == 'SP'
df.loc[filtro_graves & filtro_SP]   # Usamos o operador & para juntar os dois


,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista


In [45]:
# pesquisar ocorrências do ano de 2015
filtro_ocorrencias = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro_ocorrencias]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
1870,52992,52992,52992,52992,52992,INCIDENTE GRAVE,***,***,SALVADOR,BA,...,2015-01-01,11:40:00,***,FINALIZADA,IG-001/CENIPA/2015,SIM,2018-11-07,3,2,NÃO
1871,52979,52979,52979,52979,52979,ACIDENTE,-22.3855555556,-53.8763888889,IVINHEMA,MS,...,2015-01-02,14:50:00,***,FINALIZADA,A-002/CENIPA/2015,SIM,2018-06-25,0,1,NÃO
1872,53073,53073,53073,53073,53073,INCIDENTE,NaN,NaN,TEFÉ,AM,...,2015-01-02,16:28:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1873,53074,53074,53074,53074,53074,INCIDENTE,NaN,NaN,BELO HORIZONTE,MG,...,2015-01-02,20:34:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1874,52976,52976,52976,52976,52976,ACIDENTE,\t-24.75333333\t,\t-53.635\t,TOLEDO,PR,...,2015-01-04,22:04:00,SIM,FINALIZADA,A-003/CENIPA/2015,SIM,2016-06-16,6,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2336,60632,60632,60632,60632,60632,INCIDENTE GRAVE,-23.8441666667,-49.0569444444,ITABERÁ,SP,...,2015-12-24,14:00:00,NaN,FINALIZADA,IG-171/CENIPA/2015,SIM,2017-08-01,0,1,NÃO
2337,60600,60600,60600,60600,60600,INCIDENTE,-5.7688888889,-35.3663888889,GUARULHOS,SP,...,2015-12-25,19:00:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2338,60642,60642,60642,60642,60642,INCIDENTE,\t-26.21916667\t,\t-48.50027778\t,SÃO FRANCISCO DO SUL,SC,...,2015-12-26,16:00:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2339,60631,60631,60631,60631,60631,ACIDENTE,\t-29.1625\t,\t-55.93888889\t,MAÇAMBARÁ,RS,...,2015-12-28,19:00:00,SIM,FINALIZADA,A-172/CENIPA/2015,SIM,2018-07-27,2,1,NÃO


In [46]:
# pesquisar ocorrências do ano de 2015 e mês 12
filtro_ocorrencias = df.ocorrencia_dia.dt.year == 2015
filtro_ocorrencias_mes = df.ocorrencia_dia.dt.month == 12
df.loc[filtro_ocorrencias & filtro_ocorrencias_mes]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
2302,53573,53573,53573,53573,53573,INCIDENTE,-23.4355555556,-46.4730555556,GUARULHOS,SP,...,2015-12-01,02:48:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2303,60601,60601,60601,60601,60601,INCIDENTE,-10.29,-48.3577777778,PALMAS,TO,...,2015-12-01,16:05:00,NaN,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2304,53634,53634,53634,53634,53634,INCIDENTE,-10.29,-48.3577777778,PALMAS,TO,...,2015-12-02,17:45:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2305,53636,53636,53636,53636,53636,INCIDENTE,-23.1747222222,-46.9455555556,JUNDIAÍ,SP,...,2015-12-02,17:42:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2306,53575,53575,53575,53575,53575,INCIDENTE,-22.0366666667,-41.0708333333,CAMPOS DOS GOYTACAZES,RJ,...,2015-12-03,10:50:00,NaN,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2307,60637,60637,60637,60637,60637,INCIDENTE,-19.91,-43.9902777778,BELO HORIZONTE,MG,...,2015-12-03,16:47:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2308,53625,53625,53625,53625,53625,ACIDENTE,-16.6533333333,-49.6094444444,TRINDADE,GO,...,2015-12-06,13:10:00,NaN,FINALIZADA,A-159/CENIPA/2015,SIM,2019-04-08,3,1,NÃO
2309,53626,53626,53626,53626,53626,ACIDENTE,-22.7555555556,-47.2672222222,AMERICANA,SP,...,2015-12-06,15:00:00,NaN,FINALIZADA,A-160/CENIPA/2015,SIM,2019-10-28,1,1,SIM
2310,53628,53628,53628,53628,53628,ACIDENTE,-22.0327777778,-47.0258333333,AGUAÍ,SP,...,2015-12-08,14:30:00,NaN,FINALIZADA,A-161/CENIPA/2015,SIM,2018-05-17,1,1,NÃO
2311,53629,53629,53629,53629,53629,INCIDENTE GRAVE,-20.2991666667,-50.5469444444,JALES,SP,...,2015-12-08,10:20:00,NaN,FINALIZADA,A-162/CENIPA/2015,NÃO,NaN,0,1,NÃO


In [47]:
# pesquisar ocorrências do ano de 2015 e mês 12 e dia 8
filtro_ocorrencias = df.ocorrencia_dia.dt.year == 2015
filtro_ocorrencias_mes = df.ocorrencia_dia.dt.month == 12
filtro_ocorrencias_dia  = df.ocorrencia_dia.dt.day == 8
df.loc[filtro_ocorrencias & filtro_ocorrencias_mes & filtro_ocorrencias_dia]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
2310,53628,53628,53628,53628,53628,ACIDENTE,-22.0327777778,-47.0258333333,AGUAÍ,SP,...,2015-12-08,14:30:00,NaN,FINALIZADA,A-161/CENIPA/2015,SIM,2018-05-17,1,1,NÃO
2311,53629,53629,53629,53629,53629,INCIDENTE GRAVE,-20.2991666667,-50.5469444444,JALES,SP,...,2015-12-08,10:20:00,NaN,FINALIZADA,A-162/CENIPA/2015,NÃO,NaN,0,1,NÃO
2312,53631,53631,53631,53631,53631,INCIDENTE,-23.0069444444,-47.1344444444,CAMPINAS,SP,...,2015-12-08,16:19:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2313,60636,60636,60636,60636,60636,INCIDENTE,\t-29.19555556\t,\t-51.18972222\t,CAXIAS DO SUL,RS,...,2015-12-08,13:00:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO


In [48]:
# pesquisar ocorrências do ano de 2015 e mês 12 e do dia 3 ao dia 8
filtro_ocorrencias = df.ocorrencia_dia.dt.year == 2015
filtro_ocorrencias_mes = df.ocorrencia_dia.dt.month == 12
filtro_ocorrencias_dia_inicio  = df.ocorrencia_dia.dt.day >2 
filtro_ocorrencias_dia_fim = df.ocorrencia_dia.dt.day <9
df.loc[filtro_ocorrencias & filtro_ocorrencias_mes & filtro_ocorrencias_dia_inicio & filtro_ocorrencias_dia_fim]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
2306,53575,53575,53575,53575,53575,INCIDENTE,-22.0366666667,-41.0708333333,CAMPOS DOS GOYTACAZES,RJ,...,2015-12-03,10:50:00,NaN,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2307,60637,60637,60637,60637,60637,INCIDENTE,-19.91,-43.9902777778,BELO HORIZONTE,MG,...,2015-12-03,16:47:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2308,53625,53625,53625,53625,53625,ACIDENTE,-16.6533333333,-49.6094444444,TRINDADE,GO,...,2015-12-06,13:10:00,NaN,FINALIZADA,A-159/CENIPA/2015,SIM,2019-04-08,3,1,NÃO
2309,53626,53626,53626,53626,53626,ACIDENTE,-22.7555555556,-47.2672222222,AMERICANA,SP,...,2015-12-06,15:00:00,NaN,FINALIZADA,A-160/CENIPA/2015,SIM,2019-10-28,1,1,SIM
2310,53628,53628,53628,53628,53628,ACIDENTE,-22.0327777778,-47.0258333333,AGUAÍ,SP,...,2015-12-08,14:30:00,NaN,FINALIZADA,A-161/CENIPA/2015,SIM,2018-05-17,1,1,NÃO
2311,53629,53629,53629,53629,53629,INCIDENTE GRAVE,-20.2991666667,-50.5469444444,JALES,SP,...,2015-12-08,10:20:00,NaN,FINALIZADA,A-162/CENIPA/2015,NÃO,NaN,0,1,NÃO
2312,53631,53631,53631,53631,53631,INCIDENTE,-23.0069444444,-47.1344444444,CAMPINAS,SP,...,2015-12-08,16:19:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2313,60636,60636,60636,60636,60636,INCIDENTE,\t-29.19555556\t,\t-51.18972222\t,CAXIAS DO SUL,RS,...,2015-12-08,13:00:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO


*Para poder fazer um filtro onde eu possa obter informações do ano, mês, dia e hora de determinada ocorrência a melhor opção é criar uma nova coluna onde eu junto dia e hora, no entando algumas funções foram adiconadas (pd.to_datetime) para converter tudo no formato date e (.astype(str)) para conseguir concatenar a data e a hora.

In [ ]:
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' / ' + df.ocorrencia_hora)

In [49]:
df.head(2)

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,52242,52242,52242,52242,52242,INCIDENTE,NaN,NaN,PORTO ALEGRE,RS,...,2012-01-05,20:27:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,45331,45331,45331,45331,45331,ACIDENTE,-23.4355555556,-46.4730555556,GUARULHOS,SP,...,2012-01-06,13:44:00,SIM,FINALIZADA,A-582/CENIPA/2014,SIM,2016-09-01,3,1,NÃO


In [ ]:
# pesquisar ocorrências do ano de 2015 e mês 12 e do dia 3 ao dia 8 mostrando dia e hora
filtro_ocorrencias = df.ocorrencia_dia_hora.dt.year == 2015
filtro_ocorrencias_mes = df.ocorrencia_dia_hora.dt.month == 12
filtro_ocorrencias_dia_inicio  = df.ocorrencia_dia_hora.dt.day >2 
filtro_ocorrencias_dia_fim = df.ocorrencia_dia_hora.dt.day <9
df.loc[filtro_ocorrencias & filtro_ocorrencias_mes & filtro_ocorrencias_dia_inicio & filtro_ocorrencias_dia_fim]

## Group By  (Data Aggregating)

### *Agrupamentos de dados para gerar novas informações*

In [51]:
# Grupo de ocorrências do ano de 2015 e mês 03
filtro_ocorrencias = df.ocorrencia_dia.dt.year == 2015
filtro_ocorrencias_mes = df.ocorrencia_dia.dt.month == 3
df201503 = df.loc[filtro_ocorrencias & filtro_ocorrencias_mes]
df201503

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
1960,53183,53183,53183,53183,53183,ACIDENTE,-23.0819444444,-52.7652777778,AMAPORÃ,PR,...,2015-03-02,23:00:00,***,FINALIZADA,A-031/CENIPA/2015,NÃO,NaN,0,1,NÃO
1961,53120,53120,53120,53120,53120,ACIDENTE,-0.2002777778,-49.9672222222,CHAVES,PA,...,2015-03-04,13:30:00,***,FINALIZADA,A-032/CENIPA/2015,SIM,2015-10-19,0,1,NÃO
1962,53109,53109,53109,53109,53109,ACIDENTE,-20.5994444444,-54.6025,CAMPO GRANDE,MS,...,2015-03-05,13:50:00,SIM,FINALIZADA,A-033/CENIPA/2015,SIM,2018-09-24,0,1,NÃO
1963,53112,53112,53112,53112,53112,ACIDENTE,-22.2036111111,-47.1966666667,MOGI GUAÇU,SP,...,2015-03-06,21:00:00,***,FINALIZADA,A-034/CENIPA/2015,SIM,2021-09-17,1,1,NÃO
1964,53152,53152,53152,53152,53152,INCIDENTE GRAVE,-22.9938888889,-43.3752777778,RIO DE JANEIRO,RJ,...,2015-03-10,11:30:00,***,FINALIZADA,IG-035/CENIPA/2015,SIM,2016-08-26,2,1,NÃO
1965,53167,53167,53167,53167,53167,INCIDENTE,NaN,NaN,MARABÁ,PA,...,2015-03-10,17:33:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1966,53596,53596,53596,53596,53596,INCIDENTE,\t-27.13638889\t,\t-48.68166667\t,ITAPEMA,SC,...,2015-03-10,21:35:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1967,53149,53149,53149,53149,53149,ACIDENTE,-4.2597222222,-69.9169444444,TABATINGA,AM,...,2015-03-11,23:25:00,SIM,FINALIZADA,A-036/CENIPA/2015,SIM,2016-10-14,0,1,NÃO
1968,53148,53148,53148,53148,53148,ACIDENTE,-22.3388888889,-47.3580555556,ARARAS,SP,...,2015-03-12,12:40:00,***,FINALIZADA,A-037/CENIPA/2015,SIM,2015-10-19,0,1,NÃO
1969,53153,53153,53153,53153,53153,INCIDENTE,NaN,NaN,PORTO ALEGRE,RS,...,2015-03-13,15:45:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO


In [52]:
df201503.count()

codigo_ocorrencia                 37
codigo_ocorrencia1                37
codigo_ocorrencia2                37
codigo_ocorrencia3                37
codigo_ocorrencia4                37
ocorrencia_classificacao          37
ocorrencia_latitude               23
ocorrencia_longitude              23
ocorrencia_cidade                 37
ocorrencia_uf                     37
ocorrencia_pais                   37
ocorrencia_aerodromo              21
ocorrencia_dia                    37
ocorrencia_hora                   37
investigacao_aeronave_liberada    37
investigacao_status               35
divulgacao_relatorio_numero       35
divulgacao_relatorio_publicado    37
divulgacao_dia_publicacao         16
total_recomendacoes               37
total_aeronaves_envolvidas        37
ocorrencia_saida_pista            37
dtype: int64

In [53]:
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

In [54]:
df201503.groupby(['ocorrencia_classificacao']).size()  #contar os registros agrupados

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
dtype: int64

In [55]:
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count().sort_values()

ocorrencia_classificacao
INCIDENTE GRAVE     5
ACIDENTE           15
INCIDENTE          17
Name: codigo_ocorrencia, dtype: int64

In [56]:
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count().sort_values(ascending=False)

ocorrencia_classificacao
INCIDENTE          17
ACIDENTE           15
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

### Agrupamento dos dados das ocorrências na região Sudeste no ano de 2010

In [57]:
filtro_ocorrencias_ano = df.ocorrencia_dia.dt.year == 2010
filtro_ocorrencias_uf = df.ocorrencia_uf.isin(['SP','MG','ES','RJ'])
dfSudeste2010 = df.loc[filtro_ocorrencias_ano & filtro_ocorrencias_uf]
dfSudeste2010

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista


In [58]:
# Quantos incidentes, acidentes e incidentes graves eu tenho na Região Sudeste?

dfSudeste2010.groupby(['ocorrencia_classificacao']).size()

Series([], dtype: int64)

In [59]:
# Quantos incidentes, acidentes e incidentes graves eu tenho POR ESATDO (UF) na Região Sudeste?
dfSudeste2010.groupby(['ocorrencia_classificacao','ocorrencia_uf']).size()

Series([], dtype: int64)

In [60]:
# Quantos incidentes, acidentes e incidentes graves eu tenho POR CIDADE na Região Sudeste?
dfSudeste2010.groupby(['ocorrencia_classificacao','ocorrencia_cidade']).size().sort_values(ascending=False)

Series([], dtype: int64)

In [61]:
# Qual foi o total de recomendações por aerodromo do Rio de Janeiro?

filtro_RJ = dfSudeste2010.ocorrencia_cidade == 'RIO DE JANEIRO'
dfSudeste2010.loc[filtro_RJ].total_recomendacoes.sum()

0

In [62]:
# Qual foi o total de recomendações por cidades da Região Sudeste?

dfSudeste2010.groupby(['ocorrencia_cidade']).total_recomendacoes.sum()


Series([], Name: total_recomendacoes, dtype: int64)

In [63]:
# Total de ocorrencias POR AERODROMO na Região Sudeste

dfSudeste2010.groupby(['ocorrencia_aerodromo'], dropna=False).total_recomendacoes.sum()

Series([], Name: total_recomendacoes, dtype: int64)